In [1]:
import pandas as pd
!pwd

/c/Users/Mason Carnahan/Documents/Bootcamp/Project_1/Project_1


In [2]:
import pandas as pd
import os 
from matplotlib import pyplot as plt
import numpy as np



#Decide what columns we need
A_csv = pd.read_csv('xaa.csv')
B_csv = pd.read_csv('xab.csv', header=None)
C_csv = pd.read_csv('xac.csv', header=None)
D_csv = pd.read_csv('xad.csv', header=None)
#Add correct index to split csv
B_csv.columns = A_csv.columns
C_csv.columns = A_csv.columns
D_csv.columns = A_csv.columns
#Merging
AB_csv = pd.merge(A_csv, B_csv,how='outer')

ABC_csv = pd.merge(AB_csv, C_csv,how='outer')

ABCD_csv = pd.merge(ABC_csv, D_csv,how='outer')

master_df = ABCD_csv

In [3]:
#Function to change GeoLoc value
def geo_switch(geo):
    try:
        [lat, long] = str(geo[0])[1:-1].split(',')
    except TypeError:
        [lat, long] = ['NaN', 'NaN']
    except ValueError:
        [lat, long] = str(geo)[1:-1].split(',')
    
    
    return [lat, long]

In [5]:
#change the master_df using apply
master_df['GeoLocation'] = master_df['GeoLocation'].apply(lambda x: geo_switch(x))

In [7]:
#unique list of categories and states
cat_id = master_df['Category'].unique()
states = master_df['StateAbbr'].unique()
#dictionary to hold your data
df_dict = {}

#index through to grab dataframe for each category
for index in cat_id:
    category_group = master_df[master_df['Category'] == index]
#get category info for each state        
    for state in states:
        state_group =  category_group[category_group['StateAbbr'] == state]
#create keys for the df_dict and values for each state
        df_dict[index+' '+ state] = state_group

In [8]:
#Prevention dictionary for all states
prevent_dict = {}
for state in states:
#make keys the state abbr
    prevent_dict[state] = (df_dict.get('Prevention'+' '+state))

In [9]:
#Unhealthy dictionary for all states
unhealthy_dict = {}
for state in states: 
#make keys the state abbr
    unhealthy_dict[state] = (df_dict.get('Unhealthy Behaviors'+' '+state))

In [10]:
#Outcomes dictionary for all states
outcomes_dict = {}
for state in states:
#make keys the state abbr
    outcomes_dict[state] = (df_dict.get('Health Outcomes'+' '+state))